In [1]:
import pandas as pd
import numpy as np
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")
import statistics as stat

In [2]:
data = pd.read_csv('scdb-20rfq-60cap-metrics.csv', names = ['Event', 'Time', 'Tx_type', 'Tx_id', 'Accept_id', 'time'])

In [3]:
grouped_data = data.groupby('Tx_id')
tx_id = list(grouped_data.groups.keys())
len(tx_id)

1538

In [4]:
#new data frame to get the count of each transaction type
data_unique_ids = data.drop_duplicates(subset = ['Tx_id'])

accept_count = 0
accept_ids = []
for i in range(0, len(data_unique_ids)):
    if data_unique_ids.iloc[i]['Tx_type'] == 'ACCEPT':
        accept_ids.append(data_unique_ids.iloc[i]['Tx_id'])
        accept_count += 1   


print(f"#ACCEPTs =  {accept_count}")

#ACCEPTs =  20


In [5]:
df_test = grouped_data.get_group("331ca77987ad25091c086a5a8534dc766be3c41bcc3803fb829a498f0a7a168e")
df_test

KeyError: '331ca77987ad25091c086a5a8534dc766be3c41bcc3803fb829a498f0a7a168e'

## without RETURN 

In [6]:
time_all_committed_accepts = {}
time_passed_accepts = []
validated_time = []
accepted_time = []
committed_time = []

accept_underfit = []
accept_underfit_dict = {}

accept_overfit = []
accept_overfit_dict = {}

for i in range(0, len(tx_id)):
    temp_list = []
    if "ACCEPT" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_accepts[tx_id[i]] = actual_time
            time_passed_accepts.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
            
            check_time = df[df['Event'] == 'check_tx']['Time'].to_list()[0]
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
        elif len(temp_list) < 6:
            accept_underfit.append(temp_list)
            accept_underfit_dict[tx_id[i]] = temp_list
        elif 7 >= len(temp_list) > 6 :
            df_check = grouped_data.get_group(tx_id[i])
            if len(df_check[df_check['Event'] == 'check_tx']['Time'].to_list()) > 1:
                check_time = max(df_check[df_check['Event'] == 'check_tx']['Time'].to_list())
            else:
                check_time = df_check[df_check['Event'] == 'check_tx']['Time'].to_list()[0] 
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_accepts[tx_id[i]] = actual_time
            time_passed_accepts.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
           
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
            
            accept_overfit.append(actual_time)
            accept_overfit_dict[tx_id[i]] = temp_list

## with RETURNs

In [22]:


accept_return_count = {}

time_id_return = {}
for i in range(0, len(tx_id)):
    temp_list = []
    return_count = 0
    if "ACCEPT" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        for j in range(0, len(data)):
            if data['Event'][j] == "commit_tx" and tx_id[i] == data['Accept_id'][j]:
                temp_list.append(data["Time"][j])
                return_count += 1
        accept_return_count[tx_id[i]] = return_count
        

In [ ]:
accept_return_count

In [7]:
time_all_committed_accepts

{'01a8c04001b8a2b130c3466dca1e55c68b683aa8f3f55e2af062f0351b33b216': 207,
 '19f6ece27dce7909ecf314dbbf25ea83d097184eb9b4d5a9b0fbe2c642c96433': 197,
 '2599a2aa663dde584091f55c6873d0dad4d72740cef22a98ff5bfd1de1d37262': 200,
 '337c781bf1b00b1788f379860be0f548c62dac8795ab9ca238390f81f7528d2c': 183,
 '3e1b95d5f516967275d039023fa87d77a4550243eaf94651f227326b8ab13c4e': 217,
 '517d8cb41c5f51439e9dc3f31d8391ab6da0ceb46098037a4f4315cb37f22e3f': 391,
 '5ade1fb7b29bfb09f0e1627d307ab8ebdd0aa459c54da0542f798f2d0c9fe9a6': 338,
 '60611b0f9b3f79b11e6526b21ad8b971d5f3c834902cb34bacf9d6e68301bca9': 185,
 '6eb117fba67ce0824d20ad01e0092e6b6ae29d8352f90a95528213721311a7b2': 337,
 '6f8373907216cf129867e4cc4a5468929ee7803b3f5a68870f1e10c024ee4737': 181,
 '82c17dd6ba9c0342bf104fc126a0c5dae8f49579499e47e334d46096470a8072': 730,
 '864c3f08cd170291686f1bc791da3c1436928b3f16da5da876d9eb5fdc20a3db': 695,
 '92f3111d0b7ea2e62804343583cf5ba713b94d1b23c169504c3be01eaf51689c': 368,
 '97ac4af36d096aaa1cd2f437581805897b21

In [8]:
len(time_passed_accepts)

20

In [9]:
len(accept_overfit)

10

In [10]:
stat.mean(time_passed_accepts)

319.25

In [11]:
stat.mean(accept_overfit)

387.8

In [12]:
# Analysis of CREATE Transaction
accept_mean = "{:.2f}".format(stat.mean(time_passed_accepts))
accept_median = stat.median(time_passed_accepts)
accept_max = max(time_passed_accepts)
accept_min = min(time_passed_accepts)
accept_std_dev = "{:.2f}".format(stat.stdev(time_passed_accepts))

print(accept_mean)
print(accept_median)
print(accept_max)
print(accept_min)
print(accept_std_dev)
print(f"total accepts = {len(time_passed_accepts)}")

accept_dict = {
               'min': accept_min,
               'max': accept_max,
               'average': accept_mean,
               'median' : accept_median,
               'standard_deviation' : accept_std_dev,
               '# of txs' : len(time_passed_accepts)}

319.25
335.5
730
181
155.27
total accepts = 20


In [13]:
results = pd.DataFrame(accept_dict, index = ['ACCEPT'])
results.to_csv('accept_analysis.csv', sep=',', encoding='utf-8')
results

,min,max,average,median,standard_deviation,# of txs
ACCEPT,181,730,319.25,335.5,155.27,20
